In [47]:
import pandas as pd
df = pd.read_csv("./股價資料.csv")
df["volume"] = df["volume"].apply(lambda x : float(x.replace(",","")))

print(df.head())

         date  price_open  price_close   volume
0  2023/06/01       550.0        551.0  24927.0
1  2023/06/02       559.0        562.0  31324.0
2  2023/06/05       560.0        555.0  17229.0
3  2023/06/06       554.0        560.0  20632.0
4  2023/06/07       561.0        568.0  28668.0


In [48]:
#define "yesterday closing price" 、"yesterday volume"
df["yes_close"] = df["price_close"].shift(periods = 1)
df["yes_volume"] = df["volume"].shift(periods = 1)


#define"gain" =(today closing price) - (yesterday closing price)
df["gain"] = df["price_close"]-df["yes_close"]


#define "Up" "Up"==1 if "gain">0  "Up"==0 if "gain"<0 
df["Up"] = df["gain"].apply(lambda x : 1 if x > 0 else 0)


df = df.dropna()
df = df.reset_index(drop = True)
print(df.head())

         date  price_open  price_close   volume  yes_close  yes_volume  gain  \
0  2023/06/02       559.0        562.0  31324.0      551.0     24927.0  11.0   
1  2023/06/05       560.0        555.0  17229.0      562.0     31324.0  -7.0   
2  2023/06/06       554.0        560.0  20632.0      555.0     17229.0   5.0   
3  2023/06/07       561.0        568.0  28668.0      560.0     20632.0   8.0   
4  2023/06/08       562.0        559.0  23666.0      568.0     28668.0  -9.0   

   Up  
0   1  
1   0  
2   1  
3   1  
4   0  


In [51]:
df_normal = pd.DataFrame()
df_normal["volume_ratio"] = df["volume"].apply(pd.to_numeric)  / df["yes_volume"].apply(pd.to_numeric)
df_normal["close_price_ratio"] = df["price_close"].apply(pd.to_numeric) / df["yes_close"].apply(pd.to_numeric)

df_normal["target"] = df["Up"].shift(periods = -1)

df_normal = df_normal.dropna()
df_normal = df_normal.reset_index(drop=True)
print(df_normal.head())

   volume_ratio  close_price_ratio  target
0      1.256629           1.019964     0.0
1      0.550026           0.987544     1.0
2      1.197516           1.009009     1.0
3      1.389492           1.014286     0.0
4      0.825520           0.984155     1.0


In [52]:
X=df_normal[["volume_ratio", "close_price_ratio"]]
y=df_normal["target"]

In [53]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import warnings
#Block warning messages
warnings.filterwarnings("ignore") 


x_train,x_valid,y_train,y_valid = train_test_split(X,y,test_size=0.2,random_state=42)

#Classification model
lg = LogisticRegression()
svm = SVC(probability= True)
nb = GaussianNB()
dt = DecisionTreeClassifier(random_state = 42)
rf = RandomForestClassifier(random_state = 100)
ada = AdaBoostClassifier()
per =  Perceptron()


lg.fit(x_train,y_train)
y_pred_lg = lg.predict(x_valid)
print(f'valid score of logistic:{accuracy_score(y_valid,y_pred_lg)}')
svm.fit(x_train,y_train)
y_pred_svm = svm.predict(x_valid)
print(f'valid score of SVM:{accuracy_score(y_valid,y_pred_svm)}')
nb.fit(x_train,y_train)
y_pred_nb = nb.predict(x_valid)
print(f'valid score of naive bayes:{accuracy_score(y_valid,y_pred_nb)}')
dt.fit(x_train,y_train)
y_pred_dt = dt.predict(x_valid)
print(f'valid score of DecisionTree:{accuracy_score(y_valid,y_pred_dt)}')
rf.fit(x_train,y_train)
y_pred_rf = rf.predict(x_valid)
print(f'valid score of RandomForest:{accuracy_score(y_valid,y_pred_rf)}')
ada.fit(x_train,y_train)
y_pred_ada = ada.predict(x_valid)
print(f'valid score of AdaBoost:{accuracy_score(y_valid,y_pred_ada)}')
per.fit(x_train,y_train)
y_pred_per = per.predict(x_valid)
print(f'valid score of Perceptron:{accuracy_score(y_valid,y_pred_per)}')

valid score of logistic:0.4642857142857143
valid score of SVM:0.5
valid score of naive bayes:0.4642857142857143
valid score of DecisionTree:0.5
valid score of RandomForest:0.4642857142857143
valid score of AdaBoost:0.5
valid score of Perceptron:0.4642857142857143


In [54]:
#confusion_matrix
print(f'confusion matrix of logistic:\n{confusion_matrix(y_valid,y_pred_lg)}\n'+'-'*100)
print(f'confusion matrix of SVM:\n{confusion_matrix(y_valid,y_pred_svm)}\n'+'-'*100)
print(f'confusion matrix of naive bayes:\n{confusion_matrix(y_valid,y_pred_nb)}\n'+'-'*100)
print(f'confusion matrix of DecisionTree:\n{confusion_matrix(y_valid,y_pred_dt)}\n'+'-'*100)
print(f'confusion matrix of RandomForest:\n{confusion_matrix(y_valid,y_pred_rf)}\n'+'-'*100)
print(f'confusion matrix of AdaBoost:\n{confusion_matrix(y_valid,y_pred_ada)}\n'+'-'*100)
print(f'confusion matrix of Perceptron:\n{confusion_matrix(y_valid,y_pred_per)}\n'+'-'*100)

Label:
0.0    13
1.0    15
Name: target, dtype: int64
----------------------------------------------------------------------------------------------------
confusion matrix of logistic:
[[13  0]
 [15  0]]
----------------------------------------------------------------------------------------------------
confusion matrix of SVM:
[[13  0]
 [14  1]]
----------------------------------------------------------------------------------------------------
confusion matrix of naive bayes:
[[12  1]
 [14  1]]
----------------------------------------------------------------------------------------------------
confusion matrix of DecisionTree:
[[7 6]
 [8 7]]
----------------------------------------------------------------------------------------------------
confusion matrix of RandomForest:
[[7 6]
 [9 6]]
----------------------------------------------------------------------------------------------------
confusion matrix of AdaBoost:
[[8 5]
 [9 6]]
----------------------------------------------------

In [55]:
from sklearn.metrics import precision_score

print(f'valid precision score of logistic:{precision_score(y_valid,y_pred_lg, zero_division=1)}')
print(f'valid precision score of SVM:{precision_score(y_valid,y_pred_svm, zero_division=1)}')
print(f'valid precision score of naive bayes:{precision_score(y_valid,y_pred_nb)}')
print(f'valid precision score of DecisionTree:{precision_score(y_valid,y_pred_dt)}')
print(f'valid precision score of RandomForest:{precision_score(y_valid,y_pred_rf)}')
print(f'valid precision score of AdaBoost:{precision_score(y_valid,y_pred_ada)}')
print(f'valid precision score of Perceptron:{precision_score(y_valid,y_pred_per, zero_division=1)}')

valid precision score of logistic:1.0
valid precision score of SVM:1.0
valid precision score of naive bayes:0.5
valid precision score of DecisionTree:0.5384615384615384
valid precision score of RandomForest:0.5
valid precision score of AdaBoost:0.5454545454545454
valid precision score of Perceptron:1.0


In [56]:
from sklearn.metrics import recall_score
print(f'valid recall score of logistic:{recall_score(y_valid,y_pred_lg)}')
print(f'valid recall score of SVM:{recall_score(y_valid,y_pred_svm)}')
print(f'valid recall score of naive bayes:{recall_score(y_valid,y_pred_nb)}')
print(f'valid recall score of DecisionTree:{recall_score(y_valid,y_pred_dt)}')
print(f'valid recall score of RandomForest:{recall_score(y_valid,y_pred_rf)}')
print(f'valid recall score of AdaBoost:{recall_score(y_valid,y_pred_ada)}')
print(f'valid recall score of Perceptron:{recall_score(y_valid,y_pred_per)}')

valid recall score of logistic:0.0
valid recall score of SVM:0.06666666666666667
valid recall score of naive bayes:0.06666666666666667
valid recall score of DecisionTree:0.4666666666666667
valid recall score of RandomForest:0.4
valid recall score of AdaBoost:0.4
valid recall score of Perceptron:0.0
